In [1]:
# %%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey
from botorch.models.gp_regression import SingleTaskGP

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

train_X = torch.rand(256, 2)
train_Y = torch.sin(train_X * 10).sum(dim=-1, keepdim=True)

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)

# rand_tensor = torch.rand(1024, 128, 128, dtype=torch.double)

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = model.posterior(X_tensor).mean.sum()
#     diff = rand_tensor - X_tensor
#     loss = (diff @ diff).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"ftol": 0, "gtol": 0, "maxiter": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()
None

/opt/anaconda3/lib/python3.9/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


         36668 function calls (33428 primitive calls) in 0.191 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.191    0.191 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:45(minimize)
        1    0.001    0.001    0.191    0.191 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/lbfgsb.py:210(_minimize_lbfgsb)
       54    0.002    0.000    0.187    0.003 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:65(_compute_if_needed)
       28    0.000    0.000    0.185    0.007 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py:231(_update_fun)
       27    0.000    0.000    0.185    0.007 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py:136(update_fun)
       27    0.000    0.000    0.185    0.007 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_differentiable_